In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

In [ ]:
file=r'/Users/lalitsachan/Dropbox/DLV1/Download Data/action_log.csv'

In [ ]:
seq_data=pd.read_csv(file)

In [ ]:
seq_data.head()

In [ ]:
max([len(t) for t in seq_data['action_sequence']])

In [ ]:
X = seq_data['action_sequence']
Y = seq_data['last_action_logged']

In [ ]:
Y

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit_transform(Y)

In [ ]:
Y = to_categorical(le.fit_transform(Y))

In [ ]:
Y

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
max_len = 14
tok = Tokenizer(char_level=True,split='')
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)

In [ ]:
X_train

In [ ]:
sequences

In [ ]:
tok.index_word

In [ ]:
sequences_matrix_train = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix_train

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(9,9,input_length=14,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(8,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,
                                               maxlen=max_len)

In [ ]:
model.fit(sequences_matrix_train,Y_train,batch_size=10000,epochs=5,
          validation_data=(test_sequences_matrix,Y_test))

In [ ]:
predictions=model.predict(test_sequences_matrix)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(Y_test,predictions)

In [ ]:
# although looking at auc score ( average across all classes ) is not the best performance measure 
# classification report gives a much better idea 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
Y_test_pred=np.argmax(predictions,1)

In [ ]:
print(classification_report(np.argmax(Y_test,1),Y_test_pred))

In [ ]:
le.transform(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

In [ ]:
# As you can see that the model is actually only making prediciton for just two majority classes